In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.functions import window

import datetime
import math


In [2]:
# spark = SparkSession.builder.master("local").\
# 192.168.144.202
# spark = SparkSession.builder.master("spark://192.168.144.202:7077").\
spark = SparkSession.builder.master("local").\
                appName("Home_Work_py").\
                config("spark.driver.bindAddress", "localhost").\
                config("spark.ui.port", "4040").\
                getOrCreate()

22/11/28 17:16:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/28 17:16:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Попробовать без схемы данных

In [3]:
data = [ ('Петя','','Петров','12-12-2010','M',3000)
        ,('Маша','Михайловна','Мишина','10-09-2000','F',5000)
        ,('Коля','Николаевич','','15-06-1999','M',4000)
        ,('Вася','','Пупкин','01-01-1954','M',7000)
        ,('Иоган','Себастьян','Бах','30-05-1754','M',5000)
]

In [4]:
columns = ['firstame', 'midlename', 'secondname', 'dob', 'gender', 'salary']
df = spark.createDataFrame(data = data, schema = columns )

In [5]:
df.count()

5

In [6]:
df.show(5)

+--------+----------+----------+----------+------+------+
|firstame| midlename|secondname|       dob|gender|salary|
+--------+----------+----------+----------+------+------+
|    Петя|          |    Петров|12-12-2010|     M|  3000|
|    Маша|Михайловна|    Мишина|10-09-2000|     F|  5000|
|    Коля|Николаевич|          |15-06-1999|     M|  4000|
|    Вася|          |    Пупкин|01-01-1954|     M|  7000|
|   Иоган| Себастьян|       Бах|30-05-1754|     M|  5000|
+--------+----------+----------+----------+------+------+



In [7]:
df.printSchema()

root
 |-- firstame: string (nullable = true)
 |-- midlename: string (nullable = true)
 |-- secondname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



Мы попробуем использовать возможности Spark для анализа данных clickstream пользователей новостного Интернет-портала.

# a. Создайте схему будущего фрейма данных. 
Схема должна включать следующие атрибуты:

·id -  уникальный идентификатор посетителя сайта. Тип – последовательность чисел фиксированной длины. Данное поле не является первичным ключом.

· timestamp – дата и время события в формате unix timestamp.

· type – тип события, значение из списка (факт посещения(visit), клик по визуальному элементу страницы(click), скролл(scroll), перед на другую страницу(move)).

· page_id – id текущей страницы. Тип - последовательность чисел фиксированной длины.

· tag – каждая страница с новостью размечается редакцией специальными тегами, которые отражают тематику конкретной новости со 
страницы. Возможный список тематик: политика, спорт, медицина и т.д.

· sign – наличие у пользователя личного кабинета. Значения – True/False.

In [8]:
schema_web = T.StructType(fields=[
    T.StructField("id", T.LongType()),
    T.StructField("timestamp", T.LongType()),
    T.StructField("type", T.StringType()),
    T.StructField("page_id", T.LongType()),
    T.StructField("tag", T.StringType()),
    T.StructField("sign", T.BooleanType())
])

In [9]:
schema_lk = T.StructType(fields=[
    T.StructField("id_cab", T.LongType()),
    T.StructField("user_id", T.LongType()),
    T.StructField("FIO", T.StringType()),
    T.StructField("birthdate", T.DateType()),
    T.StructField("cab_regdate", T.StringType()),
])

# b. Создайте датафрейм 
с описанной выше схемой данных.

In [10]:
data_web = [ (1, 1667627426, "visit", 101, 'Sport', False)
            ,(1, 1667627026, "scroll", 101, 'Sport', False)
            ,(1, 1667627880, "click", 101, 'Sport', False)
            , (1, 1667627889, "click", 101, 'Sport', False)
            ,(1, 1667627980, "visit", 102, 'Politics', False)
            ,(1, 1667627180, "click", 102, 'Politics', False)
            ,(1, 1667628180, "visit", 103, 'Politics', False)
            ,(2, 1647628326, "visit", 104, 'Politics', True)
            ,(2, 1669628526, "scroll", 104, 'Politics', True)
            ,(2, 1667628680, "click", 104, 'Politics', True)
            ,(2, 1668628880, "visit", 105, 'Business', True)
            ,(2, 1667638810, "click", 105, 'Business', True)
            ,(2, 1667628819, "click", 105, 'Business', True)
            ,(3, 1667624980, "visit", 106, 'Sport', False)
            ,(3, 1667609180, "click", 106, 'Sport', False)
            ,(3, 1667629480, "visit", 107, 'Politics', False)
            ,(3, 1667029580, "scroll", 107, 'Politics', False)]

In [11]:
data_lk =[
    (101,1,"Иванов Иван Иванович", datetime.datetime(1990,7,5), datetime.datetime(2016,8,15))
    ,(102,2,"Петрова Петра Петручевна", datetime.datetime(2001,12,1), datetime.datetime(2020,4,1))
]

In [12]:
df_web = spark.createDataFrame(data=data_web, schema = schema_web)
df_lk = spark.createDataFrame(data=data_lk, schema = schema_lk)


In [13]:
df_web.show()


+---+----------+------+-------+--------+-----+
| id| timestamp|  type|page_id|     tag| sign|
+---+----------+------+-------+--------+-----+
|  1|1667627426| visit|    101|   Sport|false|
|  1|1667627026|scroll|    101|   Sport|false|
|  1|1667627880| click|    101|   Sport|false|
|  1|1667627889| click|    101|   Sport|false|
|  1|1667627980| visit|    102|Politics|false|
|  1|1667627180| click|    102|Politics|false|
|  1|1667628180| visit|    103|Politics|false|
|  2|1647628326| visit|    104|Politics| true|
|  2|1669628526|scroll|    104|Politics| true|
|  2|1667628680| click|    104|Politics| true|
|  2|1668628880| visit|    105|Business| true|
|  2|1667638810| click|    105|Business| true|
|  2|1667628819| click|    105|Business| true|
|  3|1667624980| visit|    106|   Sport|false|
|  3|1667609180| click|    106|   Sport|false|
|  3|1667629480| visit|    107|Politics|false|
|  3|1667029580|scroll|    107|Politics|false|
+---+----------+------+-------+--------+-----+



In [14]:
df_lk.show()

+------+-------+--------------------+----------+--------------------+
|id_cab|user_id|                 FIO| birthdate|         cab_regdate|
+------+-------+--------------------+----------+--------------------+
|   101|      1|Иванов Иван Иванович|1990-07-05|java.util.Gregori...|
|   102|      2|Петрова Петра Пет...|2001-12-01|java.util.Gregori...|
+------+-------+--------------------+----------+--------------------+



# c. Наполните датафрейм данными. 
Пример:
(12345, 1667627426, "click", 101, "Sport”, False)

In [15]:
data_lk_new =[
    (103,3,"Марья Ивановна Чугунова", datetime.datetime(1995,3,15), datetime.datetime(2017,5,17))
    ,(104,4,"Лёлик Боликович Козлодоев", datetime.datetime(1993,8,19), datetime.datetime(2021,12,30))
]

In [16]:
df_lk_new = spark.createDataFrame(data=data_lk_new, schema = schema_lk)
df_lk_new = df_lk.union(df_lk_new)

In [17]:
df_lk_new.show()

+------+-------+--------------------+----------+--------------------+
|id_cab|user_id|                 FIO| birthdate|         cab_regdate|
+------+-------+--------------------+----------+--------------------+
|   101|      1|Иванов Иван Иванович|1990-07-05|java.util.Gregori...|
|   102|      2|Петрова Петра Пет...|2001-12-01|java.util.Gregori...|
|   103|      3|Марья Ивановна Чу...|1995-03-15|java.util.Gregori...|
|   104|      4|Лёлик Боликович К...|1993-08-19|java.util.Gregori...|
+------+-------+--------------------+----------+--------------------+



# d.       Решите следующие задачи:

## ·   Вывести топ-5 самых активных посетителей сайта

In [18]:
# пять активных из таблицы с кликами
df_top5 = df_web.groupby("id").count().sort("count" , ascending=False)
df_top5.show(5)

+---+-----+
| id|count|
+---+-----+
|  1|    7|
|  2|    6|
|  3|    4|
+---+-----+



In [19]:
# имена пяти активных пользователей
df_top5_rez = df_top5.join(df_lk_new, on= (df_top5.id==df_lk.user_id)).select(['FIO'])
df_top5_rez.show()

+--------------------+
|                 FIO|
+--------------------+
|Иванов Иван Иванович|
|Марья Ивановна Чу...|
|Петрова Петра Пет...|
+--------------------+



## ·   Посчитать процент посетителей,
у которых есть ЛК

In [20]:
df_tmp = df_web.select(['id','sign']).distinct()
num_users = df_tmp.count()
print("Всего активных пользователей", num_users)
num_users_lk = df_tmp.select(['id','sign']).where(df_tmp.sign == True).count()
print("Всего у кого есть кабинет", num_users_lk)
print(f"Ответ: процент посетителей, у которых есть ЛК = {num_users_lk/num_users*100:.1f}%" )

Всего активных пользователей 3


Всего у кого есть кабинет 1
Ответ: процент посетителей, у которых есть ЛК = 33.3%


## ·   Вывести топ-5 страниц сайта 
по показателю общего кол-ва кликов на данной странице

In [21]:
df_web.select(["page_id", "type"]).where(df_web.type == 'click').groupby('page_id').count().sort("count", ascending=False).show(5)

+-------+-----+
|page_id|count|
+-------+-----+
|    105|    2|
|    101|    2|
|    104|    1|
|    106|    1|
|    102|    1|
+-------+-----+



## ·   Добавьте столбец к фрейму данных
со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)

In [22]:
# intrv = 4

# for i in range(0,24):
#     #h = 4
#     x = math.trunc( i / intrv )
#     xx = intrv*x
#     print(f"{i} => {xx}-{xx+3}")


In [23]:
# df_web_new = df_web.withColumn("event_time", F.from_unixtime(df_web.timestamp)
# ).withColumn("hour",F.hour( F.from_unixtime(df_web.timestamp) ) 
# ).withColumn("interval_num", 
#     F.floor( 0.25*F.hour( 
#                      F.from_unixtime(df_web.timestamp)
#                   ) 
#            ) 
# )

df_web_new = df_web.withColumn("interval_num", 
    F.floor( 0.25*F.hour( 
                     F.from_unixtime(df_web.timestamp)
                  ) 
           ) 
)

# df_web_new.show() #.printSchema() #

df_web_new0 = df_web_new.withColumn('int0', 4*df_web_new.interval_num) #.printSchema() #show()
df_web_new1 = df_web_new0.withColumn('int1', df_web_new0.int0+4) #.printSchema() #show()
# df_web_new1.show()
df_web_new_ok = df_web_new1.withColumn('event_time', F.concat(df_web_new1.int0, F.lit("-"), df_web_new1.int1)) #.printSchema() #show()
df_web_new_ok = df_web_new_ok.select(["id", "timestamp", "type", "page_id", "tag", "sign", "event_time"])
df_web_new_ok.show()

+---+----------+------+-------+--------+-----+----------+
| id| timestamp|  type|page_id|     tag| sign|event_time|
+---+----------+------+-------+--------+-----+----------+
|  1|1667627426| visit|    101|   Sport|false|       4-8|
|  1|1667627026|scroll|    101|   Sport|false|       4-8|
|  1|1667627880| click|    101|   Sport|false|       4-8|
|  1|1667627889| click|    101|   Sport|false|       4-8|
|  1|1667627980| visit|    102|Politics|false|       4-8|
|  1|1667627180| click|    102|Politics|false|       4-8|
|  1|1667628180| visit|    103|Politics|false|       4-8|
|  2|1647628326| visit|    104|Politics| true|     16-20|
|  2|1669628526|scroll|    104|Politics| true|      8-12|
|  2|1667628680| click|    104|Politics| true|       4-8|
|  2|1668628880| visit|    105|Business| true|     20-24|
|  2|1667638810| click|    105|Business| true|      8-12|
|  2|1667628819| click|    105|Business| true|       4-8|
|  3|1667624980| visit|    106|   Sport|false|       4-8|
|  3|166760918

## ·   Выведите временной промежуток 
на основе предыдущего задания, в течение которого было больше всего активностей на сайте.

In [26]:
df_web_new_ok.groupBy("event_time").count().sort("count", ascending=False).show(1)

+----------+-----+
|event_time|count|
+----------+-----+
|       4-8|   12|
+----------+-----+
only showing top 1 row



## ·   Создайте второй фрейм данных, 
который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов
1.       Id – уникальный идентификатор личного кабинета
2.       User_id – уникальный идентификатор посетителя
3.       ФИО посетителя
4.    Дату рождения посетителя 
5.       Дата создания ЛК

In [27]:
#сделано

## ·   Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.

In [28]:
df_web.show()

+---+----------+------+-------+--------+-----+
| id| timestamp|  type|page_id|     tag| sign|
+---+----------+------+-------+--------+-----+
|  1|1667627426| visit|    101|   Sport|false|
|  1|1667627026|scroll|    101|   Sport|false|
|  1|1667627880| click|    101|   Sport|false|
|  1|1667627889| click|    101|   Sport|false|
|  1|1667627980| visit|    102|Politics|false|
|  1|1667627180| click|    102|Politics|false|
|  1|1667628180| visit|    103|Politics|false|
|  2|1647628326| visit|    104|Politics| true|
|  2|1669628526|scroll|    104|Politics| true|
|  2|1667628680| click|    104|Politics| true|
|  2|1668628880| visit|    105|Business| true|
|  2|1667638810| click|    105|Business| true|
|  2|1667628819| click|    105|Business| true|
|  3|1667624980| visit|    106|   Sport|false|
|  3|1667609180| click|    106|   Sport|false|
|  3|1667629480| visit|    107|Politics|false|
|  3|1667029580|scroll|    107|Politics|false|
+---+----------+------+-------+--------+-----+



In [32]:
# пять активных из таблицы с кликами
df_sport = df_web.select(['id', 'tag']).where(df_web.tag == "Sport").distinct() #groupby("id").count().sort("count" , ascending=False)
df_sport.show(5)

+---+-----+
| id|  tag|
+---+-----+
|  1|Sport|
|  3|Sport|
+---+-----+



In [35]:
# имена пяти активных пользователей
df_sport_rez = df_sport.join(df_lk_new, on= (df_sport.id==df_lk.user_id), how='left').select(['FIO'])
df_sport_rez.show()

+--------------------+
|                 FIO|
+--------------------+
|Иванов Иван Иванович|
|Марья Ивановна Чу...|
+--------------------+



## ·   *Выведите 10% ЛК,
у которых максимальная разница между датой создания ЛК и датой последнего посещения.

## ·   *Вывести топ-5 страниц,
которые чаще всего посещают мужчины и топ-5 страниц, которые посещают чаще женщины.